In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/stanford-dogs-dataset


In [3]:
import os
import shutil
import random

source_dir = path+'/images/Images'  # Path to the folder containing breed folders
train_dir = 'structured_data/train/'
test_dir = 'structured_data/test/'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

train_ratio = 0.8

# iterating on folder of each breed
for breed in os.listdir(source_dir):
    breed_path = os.path.join(source_dir, breed)
    
    if os.path.isdir(breed_path):
        # train and test folder me breeds ka directory
        os.makedirs(os.path.join(train_dir, breed), exist_ok=True)
        os.makedirs(os.path.join(test_dir, breed), exist_ok=True)

        images = [img for img in os.listdir(breed_path)]
        
        random.shuffle(images)
        num_train = int(len(images) * train_ratio)
        
        train_images = images[:num_train]
        test_images = images[num_train:]

        for img in train_images:
            shutil.copy(os.path.join(breed_path, img), os.path.join(train_dir, breed, img))
        
        for img in test_images:
            shutil.copy(os.path.join(breed_path, img), os.path.join(test_dir, breed, img))

print("Dataset organized into train and test")

Dataset has been split and structured.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 16418 images belonging to 120 classes.
Found 4162 images belonging to 120 classes.


In [18]:
print(train_generator.image_shape)

(224, 224, 3)


In [5]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(120, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 120)                 │          61,560 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,457,656 (169.59 MB)

 Trainable params: 44,457,656 (169.59 MB)

...

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


513/513 ━━━━━━━━━━━━━━━━━━━━ 58s 94ms/step - accuracy: 0.0107 - loss: 4.9152 - val_accuracy: 0.0226 - val_loss: 4.6330
Epoch 2/10
  1/513 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.0000e+00 - loss: 4.7256

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


513/513 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 4.7256 - val_accuracy: 0.0000e+00 - val_loss: 4.6640
Epoch 3/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 42s 81ms/step - accuracy: 0.0491 - loss: 4.4415 - val_accuracy: 0.0659 - val_loss: 4.3330
Epoch 4/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 41us/step - accuracy: 0.1562 - loss: 4.0265 - val_accuracy: 0.0000e+00 - val_loss: 3.8349
Epoch 5/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 84ms/step - accuracy: 0.3299 - loss: 2.8997 - val_accuracy: 0.0550 - val_loss: 5.1497
Epoch 6/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 42us/step - accuracy: 0.3438 - loss: 2.9261 - val_accuracy: 0.5000 - val_loss: 3.2736
Epoch 7/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 84ms/step - accuracy: 0.8956 - loss: 0.4844 - val_accuracy: 0.0505 - val_loss: 9.4830
Epoch 8/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 29us/step - accuracy: 0.9375 - loss: 0.1990 - val_accuracy: 0.0000e+00 - val_loss: 11.7144
Epoch 9/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 84ms/step - accuracy: 0.9872 - loss: 0.1095 -

In [9]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test Loss: {test_loss:.2f}')
print(f'Test Accuracy: {test_acc*100:.2f} %')


130/130 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.0551 - loss: 8.2766
Test Loss: 8.42
Test Accuracy: 5.77 %


In [19]:
from tensorflow.keras import models, layers
from tensorflow.keras.applications import InceptionV3

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(120, activation='softmax') 
])

base_model.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 75s 110ms/step - accuracy: 0.5965 - loss: 1.7655 - val_accuracy: 0.7596 - val_loss: 0.8304
Epoch 2/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7812 - loss: 0.7604 - val_accuracy: 0.5000 - val_loss: 4.8638
Epoch 3/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 46s 89ms/step - accuracy: 0.8027 - loss: 0.6603 - val_accuracy: 0.7618 - val_loss: 0.8183
Epoch 4/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 91us/step - accuracy: 0.8750 - loss: 0.4631 - val_accuracy: 1.0000 - val_loss: 0.4639
Epoch 5/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 85ms/step - accuracy: 0.8369 - loss: 0.5320 - val_accuracy: 0.7675 - val_loss: 0.8710
Epoch 6/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 88us/step - accuracy: 0.7500 - loss: 0.6119 - val_accuracy: 1.0000 - val_loss: 0.1263
Epoch 7/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 45s 86ms/step - accuracy: 0.8552 - loss: 0.4330 - val_accuracy: 0.7748 - val_loss: 0.8405
Epoch 8/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s

In [24]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)

# Print the evaluation results
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')


130/130 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - accuracy: 0.7338 - loss: 1.1182
Test Loss: 1.0542
Test Accuracy: 0.7447


In [25]:
from tensorflow.keras import applications, models, layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

# Add custom layers for specific task
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(120, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_fine_tuned = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 62s 101ms/step - accuracy: 0.5917 - loss: 1.7929 - val_accuracy: 0.7553 - val_loss: 0.8621
Epoch 2/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7812 - loss: 0.6902 - val_accuracy: 1.0000 - val_loss: 0.0098
Epoch 3/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 45s 87ms/step - accuracy: 0.8018 - loss: 0.6567 - val_accuracy: 0.7738 - val_loss: 0.7753
Epoch 4/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 63us/step - accuracy: 0.8750 - loss: 0.4360 - val_accuracy: 1.0000 - val_loss: 0.0139
Epoch 5/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 44s 84ms/step - accuracy: 0.8352 - loss: 0.5228 - val_accuracy: 0.7683 - val_loss: 0.8450
Epoch 6/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/step - accuracy: 0.8125 - loss: 0.8509 - val_accuracy: 1.0000 - val_loss: 0.0472
Epoch 7/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 45s 86ms/step - accuracy: 0.8598 - loss: 0.4273 - val_accuracy: 0.7726 - val_loss: 0.8442
Epoch 8/10
513/513 ━━━━━━━━━━━━━━━━━━━━ 0s 64us/step - accuracy: 0.9688 - loss: 0.2713 - val_

In [ ]:
model.save('dog_breed_model.h5')